In [1]:
#importing required libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
#installing BeautifulSoup library to help scrape WikiPage (if not installed before)
!pip install beautifulsoup4
#installing lxml parser for BeautifulSoup to use (if not installed before)
!pip install lxml
#installing requests to work with URLs (if not installed before)
!pip install requests


In [3]:
#importing BeautifulSoup and requests
from bs4 import BeautifulSoup
import requests
#load WikiPage with list of PostalCodes for Toronto
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [5]:
#cut a table from page to work with
table = soup.find('table', class_='wikitable sortable')
#create columns out of headers in table
column_names = []
for header in table.find_all('th'):
    column_names.append(header.text.strip())
#create new dataframe with extracted columns
df = pd.DataFrame(columns=column_names)
#extract rows from table based on <tr> tag
rows = table.find_all('tr')
for row in rows[1:]: #skipping zero element as it's header
    cols = row.find_all('td')
    if cols[1].text.strip() == 'Not assigned': #skipping lines with not assigned borough
        continue
    elif cols[2].text.strip() == 'St. James Town': ##skipping duplicate St James
        continue
    elif cols[2].text.strip() == 'Not assigned': #if neighbourhood not assigned make it equal to borough
        df = df.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[1].text.strip()}], ignore_index=True)
    else:
        df = df.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[2].text.strip()}], ignore_index=True)


In [6]:
df1 = pd.read_csv('https://cocl.us/Geospatial_data')
df1.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df2 = pd.merge(df, df1, on='Postcode')
df2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [7]:
df2.shape

(208, 5)

In [8]:
toronto_df = df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
toronto_df.shape


(72, 5)

In [10]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Install and import completed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [11]:
address = 'Toronto, Ontario'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
CLIENT_ID = '0GPMCSTZ5SOR20WJQNQWRXM0YHXQDMIKAQKHMTPQM2FPHPES' #Foursquare ID
CLIENT_SECRET = 'N2VIOQ1Y45VH2MCATYXQJQZS2AEALSIC4UPVWKUGBFU0GSZS' #Foursquare Secret
VERSION = '20181020' # Foursquare API version
LIMIT = 100

Creating a function to get nearby venues for neighborhoods in Toronto

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [ ]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

In [ ]:
#checking the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,17,17,17,17,17,17
Berczy Park,55,55,55,55,55,55
Brockton,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
CN Tower,17,17,17,17,17,17
Cabbagetown,42,42,42,42,42,42
Central Bay Street,79,79,79,79,79,79
Chinatown,88,88,88,88,88,88


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 230 uniques categories.


AnalyZing Each neighborhood


In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
col_list = list(toronto_onehot.columns)
col_list.remove('Neighborhood')
fixed_columns = ['Neighborhood'] + col_list
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_onehot.shape

(3072, 230)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.000000
1,Bathurst Quay,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,CN Tower,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,...,0.00000,0.00,0.012658,0.000000,0.000000,0.012658,0.000000,0.000000,0.00,0.012658
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.034091,0.000000,0.056818,0.011364,0.000000,0.000000,0.00,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [24]:
toronto_grouped.shape

(72, 230)

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2              Café  0.04
3   Thai Restaurant  0.04
4  Sushi Restaurant  0.03


----Bathurst Quay----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4     Boat or Ferry  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3            Beer Bar  0.04
4      Farmers Market  0.04


----Brockton----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3   Grocery Store  0.05
4    Intersection  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0        Comic Shop  0.07
1     Auto Workshop  0.07
2              Park  0.07
3  Recording Studio  0.07
4        Restaurant  0.07


----CN Tower----
              venue  freq
0   Airport Service  0.18
1    Ai

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Sushi Restaurant,Bar,Gastropub,Cosmetics Shop,Asian Restaurant
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Sculpture Garden
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Farmers Market,Cheese Shop,Café,Restaurant,Japanese Restaurant
3,Brockton,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
4,Business Reply Mail Processing Centre 969 Eastern,Skate Park,Brewery,Auto Workshop,Farmers Market,Burrito Place,Restaurant,Pizza Place,Garden,Garden Center,Gym / Fitness Center
5,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Sculpture Garden
6,Cabbagetown,Coffee Shop,Italian Restaurant,Pub,Bakery,Pizza Place,Restaurant,Café,Indian Restaurant,Breakfast Spot,Japanese Restaurant
7,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Juice Bar,Japanese Restaurant,Chinese Restaurant,Café,Burger Joint,Bubble Tea Shop
8,Chinatown,Bar,Vietnamese Restaurant,Café,Bakery,Coffee Shop,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Grocery Store
9,Christie,Grocery Store,Café,Park,Restaurant,Baby Store,Diner,Athletics & Sports,Italian Restaurant,Nightclub,Coffee Shop


In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [31]:
toronto_merged = toronto_df
toronto_merged.rename(index=str, columns={"Neighbourhood":"Neighborhood"}, inplace=True)

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Café,Pub,Bakery,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Theater,Event Space
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Park,Bar,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café,Portuguese Restaurant,Chinese Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Japanese Restaurant,Burger Joint,Electronics Store,Pizza Place,Plaza,Cosmetics Shop
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Japanese Restaurant,Burger Joint,Electronics Store,Pizza Place,Plaza,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Health Food Store,Pub,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Pub,Bakery,Park,Breakfast Spot,Restaurant,Mexican Restaurant,Theater,Event Space
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Japanese Restaurant,Burger Joint,Electronics Store,Pizza Place,Plaza,Cosmetics Shop
3,Downtown Toronto,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Japanese Restaurant,Burger Joint,Electronics Store,Pizza Place,Plaza,Cosmetics Shop
4,East Toronto,0,Coffee Shop,Health Food Store,Pub,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Juice Bar,Japanese Restaurant,Chinese Restaurant,Café,Burger Joint,Bubble Tea Shop
7,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Baby Store,Diner,Athletics & Sports,Italian Restaurant,Nightclub,Coffee Shop
8,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Sushi Restaurant,Bar,Gastropub,Cosmetics Shop,Asian Restaurant
9,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Sushi Restaurant,Bar,Gastropub,Cosmetics Shop,Asian Restaurant
10,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Sushi Restaurant,Bar,Gastropub,Cosmetics Shop,Asian Restaurant
11,West Toronto,0,Pharmacy,Bakery,Grocery Store,Pool,Brewery,Café,Supermarket,Bar,Bank,Middle Eastern Restaurant


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gastropub,Bar,Seafood Restaurant,American Restaurant,Japanese Restaurant
22,West Toronto,1,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
37,Central Toronto,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Chinese Restaurant
48,Central Toronto,1,Tennis Court,Park,Playground,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
56,Central Toronto,1,Pub,Coffee Shop,Pizza Place,Supermarket,Restaurant,Fried Chicken Joint,Sports Bar,American Restaurant,Sushi Restaurant,Liquor Store


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,2,Coffee Shop,Park,Bar,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café,Portuguese Restaurant,Chinese Restaurant
5,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Farmers Market,Cheese Shop,Café,Restaurant,Japanese Restaurant
28,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Deli / Bodega,Gastropub,Seafood Restaurant
31,Central Toronto,2,Ice Cream Shop,Garden,Yoga Studio,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
34,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
43,Central Toronto,2,Sandwich Place,Pizza Place,Dessert Shop,Gym,Café,Italian Restaurant,Sushi Restaurant,Coffee Shop,Park,Pharmacy
53,Central Toronto,2,Pub,Coffee Shop,Pizza Place,Supermarket,Restaurant,Fried Chicken Joint,Sports Bar,American Restaurant,Sushi Restaurant,Liquor Store


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,West Toronto,3,Mexican Restaurant,Bar,Café,Thai Restaurant,Flea Market,Diner,Bakery,Italian Restaurant,Cajun / Creole Restaurant,Speakeasy


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,4,Tennis Court,Park,Playground,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
